## Get envirionment variables

In [430]:
import os
from dotenv import find_dotenv, dotenv_values

keys = list(dotenv_values(find_dotenv('.env')).items())
OPENAI_API_KEY = os.environ['OPENAI_API_KEY'] = keys[0][1]
LANGCHAIN_API_KEY = os.environ['LANGCHAIN_API_KEY'] = keys[1][1]
POLYGON_API_KEY = os.environ['POLYGON_API_KEY'] = keys[2][1]
EMAIL = os.environ['EMAIL'] = keys[3][1] #make this a user entry

## Install Required Libraries

In [431]:
!pip install langchain_core langchain_openai langchain_community langsmith openai tiktoken cohere lxml polygon-api-client weasyprint html5lib pydyf CFFI tinycss2 cssselect2 Pyphen Pillow fontTools pypdf -qU


[notice] A new release of pip is available: 23.1.2 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


## Preprocess S&P data

In [485]:
import pandas as pd

s_and_p = pd.read_csv("./data/S&P.csv", sep='\t', names=["No.", "Ticker", "Company Name", "Market Cap", "Stock Price", "% Change", "Revenue"])

tickers = s_and_p["Ticker"]

for ticker in tickers:
    print(ticker)

AAPL
MSFT
NVDA
GOOGL
GOOG
AMZN
META
BRK-B
LLY
TSLA
AVGO
JPM
WMT
V
XOM
UNH
MA
PG
ORCL
JNJ
COST
HD
BAC
MRK
ABBV
CVX
KO
NFLX
AMD
ADBE
CRM
PEP
LIN
TMUS
QCOM
WFC
TMO
ACN
CSCO
MCD
DHR
TXN
INTU
AMAT
ABT
AMGN
AXP
DIS
VZ
CAT
MS
IBM
GE
PFE
PM
BX
GS
CMCSA
SPGI
ISRG
NOW
UNP
INTC
NEE
HON
UBER
RTX
LOW
T
BKNG
COP
MU
PGR
C
TJX
SYK
LRCX
UPS
VRTX
BLK
ETN
ELV
REGN
ADI
LMT
PLD
BSX
BA
SCHW
NKE
MMC
CB
PANW
DE
ANET
KLAC
MDT
ADP
KKR
AMT
CI
ABNB
FI
GILD
MDLZ
WM
SO
BMY
SNPS
CRWD
ICE
SBUX
MO
HCA
MCO
ZTS
DUK
SHW
CL
GD
CDNS
CVS
APH
FDX
EQIX
EOG
CMG
TT
ITW
MCK
CTAS
CME
TGT
PH
PNC
MAR
NXPI
USB
SLB
FCX
ECL
TDG
CSX
EMR
BDX
MSI
AON
NOC
RSG
PYPL
WELL
ORLY
ROP
CARR
APD
PSX
AJG
CEG
MPC
MMM
TFC
PCAR
COF
GM
F
OXY
NEM
HLT
PSA
MET
ADSK
AFL
NSC
MNST
EW
AIG
WMB
DHI
CPRT
AZO
DLR
TRV
SPG
O
OKE
AEP
ROST
SRE
MCHP
VLO
HUM
KMB
URI
TEL
BK
GWW
SMCI
HES
CHTR
MRNA
JCI
CCI
PRU
PAYX
DXCM
KMI
GEV
LEN
STZ
LHX
FTNT
ALL
COR
KDP
D
ODFL
FIS
RCL
IQV
AMP
KHC
IDXX
AME
CMI
FAST
OTIS
MSCI
VRSK
MPWR
FICO
KR
IR
NUE
GLW
HSY
EA
DOW
A
CTVA
PCG
GEHC
FANG
P

## Set up functions to get docs from the EDGAR database

In [488]:
from weasyprint import HTML
import os

def get_CIK(ticker) -> str:
    """This tool takes a company stock ticker as an argument and returns the CIK number. This is used when trying to query the EDGAR database for financial statements."""
    if ticker is not None:
        result = CIK_df[CIK_df["ticker"] == ticker]
        cik = result["cik"]
        print(f'Ticker: {ticker}        CIK: {str(cik.item()).zfill(10)}')
        return str(cik.item()).zfill(10)
    
def get_financial_report(cik, report_type):
    """This tool takes a company CIK number from the get_CIK tool, financial report type, and an optional date. 
    This information is used to retrieve requested document from the EDGAR database and save it to a path that can be returned to the user. 
    If a date is provided, it must be in YYYY-MM-DD format."""

    url = f"https://data.sec.gov/submissions/CIK{cik}.json"
    header = {
        "User-Agent" : EMAIL
    }
    company_filings = requests.get(url, headers=header).json()
    company_filings_df = pd.DataFrame(company_filings["filings"]["recent"])
    company_filings_df = company_filings_df[company_filings_df.form == report_type]
    
    access_number = company_filings_df.accessionNumber.values[0].replace("-", "")
    file_name = company_filings_df.primaryDocument.values[0]
    pdf_path = f'./data/{report_type}/{file_name}'+".pdf"
    if not os.path.exists(pdf_path):
        url = f"https://www.sec.gov/Archives/edgar/data/{cik}/{access_number}/{file_name}"
        # dowloading and saving requested document to working directory
        req_content = requests.get(url, headers=header).content.decode("utf-8")
        print(f'Creating {pdf_path}...')
        HTML(string=req_content, base_url="").write_pdf(pdf_path)

## Loop through S&P 500 Companies and retreive docs

In [489]:
report_types = ["10-K", "10-Q"]

for ticker in tickers:
    cik = get_CIK(ticker)
    for report in report_types:
        get_financial_report(cik, report)

Ticker: AAPL        CIK: 0000320193
Ticker: MSFT        CIK: 0000789019
Ticker: NVDA        CIK: 0001045810
Ticker: GOOGL        CIK: 0001652044
Ticker: GOOG        CIK: 0001652044
Ticker: AMZN        CIK: 0001018724
Ticker: META        CIK: 0001326801
Ticker: BRK-B        CIK: 0001067983
Creating ./data/10-K/brka-20231231.htm.pdf...
Creating ./data/10-Q/brka-20240331.htm.pdf...


## Create Tools

The following tools were ripped directly from the langchain source code to remove the requirement for current data from the API.

In [432]:
"""
Util that calls several of Polygon's stock market REST APIs.
Docs: https://polygon.io/docs/stocks/getting-started
"""

import json
from typing import Any, Dict, Optional

import requests
from langchain_core.pydantic_v1 import BaseModel, root_validator
from langchain_core.utils import get_from_dict_or_env

POLYGON_BASE_URL = "https://api.polygon.io/"

class PolygonAPIWrapper(BaseModel):
    """Wrapper for Polygon API."""

    polygon_api_key: Optional[str] = None

    @root_validator(pre=True)
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that api key in environment."""
        polygon_api_key = get_from_dict_or_env(
            values, "polygon_api_key", "POLYGON_API_KEY"
        )
        values["polygon_api_key"] = polygon_api_key

        return values

    def get_financials(self, ticker: str) -> Optional[dict]:
        """
        Get fundamental financial data, which is found in balance sheets,
        income statements, and cash flow statements for a given ticker.

        /vX/reference/financials
        """
        url = (
            f"{POLYGON_BASE_URL}vX/reference/financials?"
            f"ticker={ticker}&"
            f"apiKey={self.polygon_api_key}"
        )
        response = requests.get(url)
        data = response.json()

        status = data.get("status", None)
        if status != "OK":
            raise ValueError(f"API Error: {data}")

        return data.get("results", None)

    def get_last_quote(self, ticker: str) -> Optional[dict]:
        """
        Get the most recent National Best Bid and Offer (Quote) for a ticker.

        /v2/last/nbbo/{ticker}
        """
        url = f"{POLYGON_BASE_URL}v2/last/nbbo/{ticker}?apiKey={self.polygon_api_key}"
        response = requests.get(url)
        data = response.json()

        status = data.get("status", None)
        if status != "OK":
            raise ValueError(f"API Error: {data}")

        return data.get("results", None)

    def get_ticker_news(self, ticker: str) -> Optional[dict]:
        """
        Get the most recent news articles relating to a stock ticker symbol,
        including a summary of the article and a link to the original source.

        /v2/reference/news
        """
        url = (
            f"{POLYGON_BASE_URL}v2/reference/news?"
            f"ticker={ticker}&"
            f"apiKey={self.polygon_api_key}"
        )
        response = requests.get(url)
        data = response.json()

        status = data.get("status", None)
        if status != "OK":
            raise ValueError(f"API Error: {data}")

        return data.get("results", None)

    def get_aggregates(self, ticker: str, **kwargs: Any) -> Optional[dict]:
        """
        Get aggregate bars for a stock over a given date range
        in custom time window sizes.

        /v2/aggs/ticker/{ticker}/range/{multiplier}/{timespan}/{from_date}/{to_date}
        """
        timespan = kwargs.get("timespan", "day")
        multiplier = kwargs.get("timespan_multiplier", 1)
        from_date = kwargs.get("from_date", None)
        to_date = kwargs.get("to_date", None)
        adjusted = kwargs.get("adjusted", True)
        sort = kwargs.get("sort", "asc")

        url = (
            f"{POLYGON_BASE_URL}v2/aggs"
            f"/ticker/{ticker}"
            f"/range/{multiplier}"
            f"/{timespan}"
            f"/{from_date}"
            f"/{to_date}"
            f"?apiKey={self.polygon_api_key}"
            f"&adjusted={adjusted}"
            f"&sort={sort}"
        )
        response = requests.get(url)
        data = response.json()

        status = data.get("status", None)
        if status != "DELAYED":
            raise ValueError(f"API Error: {data}")

        return data.get("results", None)

    def run(self, mode: str, ticker: str, **kwargs: Any) -> str:
        if mode == "get_financials":
            return json.dumps(self.get_financials(ticker))
        elif mode == "get_last_quote":
            return json.dumps(self.get_last_quote(ticker))
        elif mode == "get_ticker_news":
            return json.dumps(self.get_ticker_news(ticker))
        elif mode == "get_aggregates":
            return json.dumps(self.get_aggregates(ticker, **kwargs))
        else:
            raise ValueError(f"Invalid mode {mode} for Polygon API.")


In [433]:
from typing import Optional, Type

from langchain_core.callbacks import CallbackManagerForToolRun
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import BaseTool

class PolygonAggregatesSchema(BaseModel):
    """Input for PolygonAggregates."""

    ticker: str = Field(
        description="The ticker symbol to fetch aggregates for.",
    )
    timespan: str = Field(
        description="The size of the time window. "
        "Possible values are: "
        "second, minute, hour, day, week, month, quarter, year. "
        "Default is 'day'",
    )
    timespan_multiplier: int = Field(
        description="The number of timespans to aggregate. "
        "For example, if timespan is 'day' and "
        "timespan_multiplier is 1, the result will be daily bars. "
        "If timespan is 'day' and timespan_multiplier is 5, "
        "the result will be weekly bars.  "
        "Default is 1.",
    )
    from_date: str = Field(
        description="The start of the aggregate time window. "
        "Either a date with the format YYYY-MM-DD or "
        "a millisecond timestamp.",
    )
    to_date: str = Field(
        description="The end of the aggregate time window. "
        "Either a date with the format YYYY-MM-DD or "
        "a millisecond timestamp.",
    )


class PolygonAggregates(BaseTool):
    """
    Tool that gets aggregate bars (stock prices) over a
    given date range for a given ticker from Polygon.
    """

    mode: str = "get_aggregates"
    name: str = "polygon_aggregates"
    description: str = (
        "A wrapper around Polygon's Aggregates API. "
        "This tool is useful for fetching aggregate bars (stock prices) for a ticker. "
        "Input should be the ticker, date range, timespan, and timespan multiplier"
        " that you want to get the aggregate bars for. This should be done when "
        "attempting to retreive the current stock price for any valuation calculation."
    )
    args_schema: Type[PolygonAggregatesSchema] = PolygonAggregatesSchema

    api_wrapper: PolygonAPIWrapper

    def _run(
        self,
        ticker: str,
        timespan: str,
        timespan_multiplier: int,
        from_date: str,
        to_date: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the Polygon API tool."""
        return self.api_wrapper.run(
            mode=self.mode,
            ticker=ticker,
            timespan=timespan,
            timespan_multiplier=timespan_multiplier,
            from_date=from_date,
            to_date=to_date,
        )


In [434]:
from typing import Optional, Type

from langchain_core.callbacks import CallbackManagerForToolRun
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.tools import BaseTool

class Inputs(BaseModel):
    """Inputs for Polygon's Financials API"""

    query: str


class PolygonFinancials(BaseTool):
    """Tool that gets the financials of a ticker from Polygon"""

    mode: str = "get_financials"
    name: str = "polygon_financials"
    description: str = (
        "A wrapper around Polygon's Stock Financials API. "
        "This tool is useful for fetching fundamental financials from "
        "balance sheets, income statements, and cash flow statements "
        "for a stock ticker. The input should be the ticker that you want "
        "to get the latest fundamental financial data for. "
        "If a duckduckgo_search call returns a list of stocks, "
        "The tickers for those stocks can be passed to this function to "
        "retreive financial data and provide an accurate response "
        "to the user."
    )
    args_schema: Type[BaseModel] = Inputs

    api_wrapper: PolygonAPIWrapper

    def _run(
        self,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the Polygon API tool."""
        return self.api_wrapper.run(self.mode, ticker=query)


In [435]:
from typing import Optional, Type

from langchain_core.callbacks import CallbackManagerForToolRun
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.tools import BaseTool

class Inputs(BaseModel):
    """Inputs for Polygon's Ticker News API"""

    query: str


class PolygonTickerNews(BaseTool):
    """Tool that gets the latest news for a given ticker from Polygon"""

    mode: str = "get_ticker_news"
    name: str = "polygon_ticker_news"
    description: str = (
        "A wrapper around Polygon's Ticker News API. "
        "This tool is useful for fetching the latest news for a stock. "
        "Input should be the ticker that you want to get the latest news for."
    )
    args_schema: Type[BaseModel] = Inputs

    api_wrapper: PolygonAPIWrapper

    def _run(
        self,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the Polygon API tool."""
        return self.api_wrapper.run(self.mode, ticker=query)


## Set up vectorstore

In [436]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

"""
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
loader = PyPDFLoader()

if os.path.exists("./data/vectorstore"):
    vectorstore = FAISS.load_local(
        "./data/vectorstore", 
        embeddings, 
        allow_dangerous_deserialization=True # this is necessary to load the vectorstore from disk as it's stored as a `.pkl` file.
    )
    retriever = vectorstore.as_retriever()
    print("Loaded Vectorstore")
else:
    print("Indexing Files")
    os.makedirs("./data/vectorstore", exist_ok=True)
    for i in range(0, len(split_documents), 32):
        if i == 0:
            vectorstore = FAISS.from_documents(split_documents[i:i+32], embeddings)
            continue
        vectorstore.add_documents(split_documents[i:i+32])
    vectorstore.save_local("./data/vectorstore")

"""


'\nembeddings = OpenAIEmbeddings(model="text-embedding-3-small")\nloader = PyPDFLoader()\n\nif os.path.exists("./data/vectorstore"):\n    vectorstore = FAISS.load_local(\n        "./data/vectorstore", \n        embeddings, \n        allow_dangerous_deserialization=True # this is necessary to load the vectorstore from disk as it\'s stored as a `.pkl` file.\n    )\n    retriever = vectorstore.as_retriever()\n    print("Loaded Vectorstore")\nelse:\n    print("Indexing Files")\n    os.makedirs("./data/vectorstore", exist_ok=True)\n    for i in range(0, len(split_documents), 32):\n        if i == 0:\n            vectorstore = FAISS.from_documents(split_documents[i:i+32], embeddings)\n            continue\n        vectorstore.add_documents(split_documents[i:i+32])\n    vectorstore.save_local("./data/vectorstore")\n\n'

In [437]:
from langchain.tools import tool
import datetime
import requests
from langchain_community.document_loaders import UnstructuredHTMLLoader
from weasyprint import HTML

@tool
def get_datetime() -> str:
    """Get the current date and time in YYYY-MM-DD HH:MM:SS format."""
    return str(datetime.datetime.now())

@tool
def get_date() -> str:
    """Get the current date in YYYY-MM-DD format. Also useful when determining the current quarter."""
    return str(datetime.datetime.now()).split(" ")[0]

@tool
def get_time() -> str:
    """Get the current time in HH:MM:SS format."""
    return str(datetime.datetime.now()).split(" ")[1]

@tool
def get_quarter(date:str) -> str:
    """This tool takes a date in YYYY-MM-DD format as an argument and returns the quarter and year in the format 'QQ YYYY'."""
    quarters = {
        "01" : "Q1",
        "02" : "Q1",
        "03" : "Q1",
        "04" : "Q2",
        "05" : "Q2",
        "06" : "Q2",
        "07" : "Q3",
        "08" : "Q3",
        "09" : "Q3",
        "10" : "Q4",
        "11" : "Q4",
        "12" : "Q4",
    }
    return quarters[date.split("-")[1]] + f" {date.split('-')[0]}"

@tool
def store_financial_report(url):
    """This tool takes a source"""

    


## Get JSON CIK data

In [438]:
import json

#Reference found here: https://www.kaggle.com/code/svendaj/extracting-data-from-sec-edgar-restful-apis

with open("./data/company_tickers_exchange.json", "r") as f:
    CIK_dict = json.load(f)

CIK_df = pd.DataFrame(CIK_dict["data"], columns=CIK_dict["fields"])

## Set up tool belt

In [439]:
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun

api_wrapper = PolygonAPIWrapper(polygon_api_key=POLYGON_API_KEY)

tool_belt = [
    get_datetime,
    get_date,
    get_time,
    get_quarter,
    DuckDuckGoSearchRun(),
    PolygonAggregates(api_wrapper=api_wrapper),
    PolygonFinancials(api_wrapper=api_wrapper),
    PolygonTickerNews(api_wrapper=api_wrapper),
]

## Set up tool executor

In [440]:
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tool_belt)

## Set up model

In [441]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)

## Set up function calling

In [442]:
from langchain_core.utils.function_calling import convert_to_openai_function

functions = [convert_to_openai_function(t) for t in tool_belt]
model = model.bind_functions(functions)

## Set up agent state

In [443]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

## Create nodes

In [444]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

def call_model(state):
  messages = state["messages"]
  response = model.invoke(messages)
  return {"messages" : [response]}

def call_tool(state):
  last_message = state["messages"][-1]

  action = ToolInvocation(
      tool=last_message.additional_kwargs["function_call"]["name"],
      tool_input=json.loads(
          last_message.additional_kwargs["function_call"]["arguments"]
      )
  )

  response = tool_executor.invoke(action)

  function_message = FunctionMessage(content=str(response), name=action.tool)

  return {"messages" : [function_message]}

In [445]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)

workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

In [446]:
workflow.set_entry_point("agent")

In [447]:
def should_continue(state):
  last_message = state["messages"][-1]

  if "function_call" not in last_message.additional_kwargs:
    return "end"

  return "continue"

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue" : "action",
        "end" : END
    }
)

In [448]:
workflow.add_edge("action", "agent")

In [449]:
app = workflow.compile()

In [450]:
def print_messages(messages):
  next_is_tool = False
  initial_query = True
  with open("response.md", "w") as f:
    for message in messages["messages"]:
      if "function_call" in message.additional_kwargs:
        f.writelines("\n")
        f.writelines(f'Tool Call - Name: {message.additional_kwargs["function_call"]["name"]} + Query: {message.additional_kwargs["function_call"]["arguments"]}')
        print()
        print(f'Tool Call - Name: {message.additional_kwargs["function_call"]["name"]} + Query: {message.additional_kwargs["function_call"]["arguments"]}')
        next_is_tool = True
        continue
      if next_is_tool:
        print(f"Tool Response: {message.content}")
        next_is_tool = False
        continue
      if initial_query:
        f.writelines(f"Initial Query: {message.content}")
        f.writelines("\n")
        print(f"Initial Query: {message.content}")
        print()
        initial_query = False
        continue
      f.writelines("\n")
      f.writelines(f"Agent Response: {message.content}")
      print()
      print(f"Agent Response: {message.content}")

In [452]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="Can you provide a full fundamental analysis of Crowdstrike stock and use the information to determine both the stock's intrinsic and relative valuations as of today?")]}

messages = app.invoke(inputs)

print_messages(messages)


Initial Query: Can you provide a full fundamental analysis of Crowdstrike stock and use the information to determine both the stock's intrinsic and relative valuations as of today?


Tool Call - Name: get_date + Query: {}
Tool Response: 2024-07-17

Tool Call - Name: polygon_financials + Query: {"query":"CRWD"}
Tool Response: [{"start_date": "2023-05-01", "end_date": "2024-04-30", "timeframe": "ttm", "fiscal_period": "TTM", "fiscal_year": "", "cik": "0001535527", "sic": "7372", "tickers": ["CRWD"], "company_name": "CrowdStrike Holdings, Inc.", "financials": {"cash_flow_statement": {"net_cash_flow_from_financing_activities_continuing": {"value": 82732000.0, "unit": "USD", "label": "Net Cash Flow From Financing Activities, Continuing", "order": 800}, "net_cash_flow_from_financing_activities": {"value": 82732000.0, "unit": "USD", "label": "Net Cash Flow From Financing Activities", "order": 700}, "net_cash_flow_from_investing_activities": {"value": -457779000.0, "unit": "USD", "label": "Net

Initial Query: Can you provide a full fundamental analysis of Crowdstrike stock and use the information to determine both the stock's intrinsic and relative valuations as of today?

Tool Call - Name: get_date + Query: {}
Tool Call - Name: polygon_financials + Query: {"query":"CRWD"}
Tool Call - Name: polygon_aggregates + Query: {"ticker":"CRWD","timespan":"day","timespan_multiplier":1,"from_date":"2024-07-16","to_date":"2024-07-17"}
Agent Response: ### Fundamental Analysis of CrowdStrike Holdings, Inc. (CRWD)

#### Financial Overview (TTM as of April 30, 2024)
- **Revenue**: $3,284.01 million
- **Net Income**: $136.35 million
- **Gross Profit**: $2,472.52 million
- **Operating Income**: $24.40 million
- **Total Assets**: $6,841.99 million
- **Total Liabilities**: $4,273.20 million
- **Total Equity**: $2,568.78 million
- **Cash Flow from Operating Activities**: $1,248.54 million
- **Cash Flow from Investing Activities**: -$457.78 million
- **Cash Flow from Financing Activities**: $82.73 million

#### Key Ratios
- **Gross Margin**: 75.3%
- **Operating Margin**: 0.7%
- **Net Profit Margin**: 4.2%
- **Current Ratio**: 1.8
- **Quick Ratio**: 1.8
- **Debt to Equity Ratio**: 1.66
- **Return on Equity (ROE)**: 5.3%
- **Return on Assets (ROA)**: 2.0%

### Intrinsic Valuation

#### Discounted Cash Flow (DCF) Analysis
To perform a DCF analysis, we need to project future cash flows and discount them to present value. Here are the assumptions and steps:

1. **Free Cash Flow (FCF) Calculation**:
   - **FCF (TTM)**: $1,248.54 million (Cash Flow from Operating Activities) - $457.78 million (Cash Flow from Investing Activities) = $790.76 million

2. **Growth Rate Assumptions**:
   - **Short-term Growth Rate (next 5 years)**: 20% (based on historical growth and industry outlook)
   - **Long-term Growth Rate (beyond 5 years)**: 3% (terminal growth rate)

3. **Discount Rate**:
   - **Weighted Average Cost of Capital (WACC)**: 10%

4. **DCF Calculation**:
   - **Year 1-5 FCF Projections**:
     - Year 1: $790.76 million * 1.20 = $948.91 million
     - Year 2: $948.91 million * 1.20 = $1,138.69 million
     - Year 3: $1,138.69 million * 1.20 = $1,366.43 million
     - Year 4: $1,366.43 million * 1.20 = $1,639.72 million
     - Year 5: $1,639.72 million * 1.20 = $1,967.66 million

   - **Terminal Value**:
     - Terminal Value = Year 5 FCF * (1 + Terminal Growth Rate) / (WACC - Terminal Growth Rate)
     - Terminal Value = $1,967.66 million * 1.03 / (0.10 - 0.03) = $28,944.57 million

   - **Present Value of FCF**:
     - PV Year 1 = $948.91 million / (1 + 0.10)^1 = $862.64 million
     - PV Year 2 = $1,138.69 million / (1 + 0.10)^2 = $941.91 million
     - PV Year 3 = $1,366.43 million / (1 + 0.10)^3 = $1,026.68 million
     - PV Year 4 = $1,639.72 million / (1 + 0.10)^4 = $1,117.12 million
     - PV Year 5 = $1,967.66 million / (1 + 0.10)^5 = $1,213.20 million
     - PV Terminal Value = $28,944.57 million / (1 + 0.10)^5 = $17,975.68 million

   - **Total Present Value of FCF**:
     - Total PV = $862.64 million + $941.91 million + $1,026.68 million + $1,117.12 million + $1,213.20 million + $17,975.68 million = $23,137.23 million

   - **Intrinsic Value per Share**:
     - Intrinsic Value = Total PV / Shares Outstanding
     - Shares Outstanding = 737.55 million (diluted average shares)
     - Intrinsic Value per Share = $23,137.23 million / 737.55 million = $31.37

### Relative Valuation

#### Price Multiples
- **Current Stock Price**: $354.94 (as of July 17, 2024)
- **P/E Ratio**: Current Stock Price / EPS
  - EPS (TTM) = $0.53
  - P/E Ratio = $354.94 / $0.53 = 669.70

- **P/S Ratio**: Current Stock Price / Revenue per Share
  - Revenue per Share (TTM) = $3,284.01 million / 737.55 million = $4.45
  - P/S Ratio = $354.94 / $4.45 = 79.77

- **P/B Ratio**: Current Stock Price / Book Value per Share
  - Book Value per Share = $2,568.78 million / 737.55 million = $3.48
  - P/B Ratio = $354.94 / $3.48 = 102.02

### Conclusion

- **Intrinsic Valuation**: The intrinsic value of CrowdStrike's stock is estimated to be $31.37 per share based on the DCF analysis.
- **Relative Valuation**: The stock appears to be significantly overvalued based on traditional price multiples (P/E, P/S, P/B).

Given the current stock price of $354.94, CrowdStrike's stock seems to be trading at a premium, which may be driven by high growth expectations and market sentiment. Investors should consider these factors and perform further due diligence before making investment decisions.